In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Predicción de abandono en telecomunicaciones
El código de la solución se divide en las siguientes secciones:
* Comprensión de datos
* Preprocesamiento
* AED
* Manejar valores faltantes
* Ingeniería de características
* Modelo 1 - Predicción de abandono de clientes
     * Selección de características y reducción de dimensionalidad mediante PCA
     * Manejo de Desequilibrio de Clases usando ADASYN
     * Construcción de modelo de referencia
     * Validación cruzada
     * Ajuste de hiperparámetros
     * Evaluación del modelo
     * Selección de modelo
* Modelo 2: identificación de fuertes predictores de abandono (características importantes)
     * Selección de características usando ExtraTreesClassifier
     * Manejo de Desequilibrio de Clases usando ADASYN
     * Construcción del modelo
     * Ajuste de hiperparámetros con Cross Validation
     * Evaluación del modelo
    
* Recomendación de estrategia para gestionar la rotación de clientes

## PREPROCESAMIENTO


---

* Comencemos cargando las bibliotecas requeridas.
* También se inicializa la constante global RANDOM_STATE para usar en las siguientes secciones.
* Cargar el archivo del conjunto de datos (telecom_churn_data.csv) desde la carpeta de datos.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from imblearn.over_sampling import SMOTE, ADASYN

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RandomizedSearchCV, GridSearchCV

import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve

from sklearn.decomposition import PCA

%matplotlib inline
RANDOM_STATE = 42



In [ ]:
data = pd.read_csv('/content/drive/MyDrive/XLDiaz/Telecom-Churn-Prediction-master/data/telecom_churn_data.csv')
data.head().T

,0,1,2,3,4
mobile_number,7000842753,7001865778,7001625959,7001204172,7000142493
circle_id,109,109,109,109,109
loc_og_t2o_mou,0.0,0.0,0.0,0.0,0.0
std_og_t2o_mou,0.0,0.0,0.0,0.0,0.0
loc_ic_t2o_mou,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
aon,968,1006,1103,2491,1526
aug_vbc_3g,30.4,0.0,0.0,0.0,0.0
jul_vbc_3g,0.0,0.0,0.0,0.0,0.0
jun_vbc_3g,101.2,0.0,4.17,0.0,0.0


Podemos comenzar a ver algunos patrones útiles en el conjunto de datos.

* mobile_number es la columna de id único para cada cliente.
* las columnas están segregadas por meses de junio (6), julio (7), agosto (8), septiembre (9) para el año 2014.

In [ ]:
data.shape

(99999, 226)

In [ ]:
# Ahora veremos los tipos de las columnas en el conjunto de datos.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


#### FUNCIONES AUXILIARES PARA EL PREPROCESAMIENTO DE DATOS


In [ ]:
'''
Input(s)    : 1 DataFrame
Output(s)   : 6 lists  
Descripción :
- Este método lee las columnas en el marco de datos dado y las divide en varias categorías, como meses, columnas relacionadas con la fecha
   y otras columnas comunes
- Luego devuelve las listas de columnas como salida para cada una de estas categorías.
'''
def get_cols_split(df):

    col_len = len(df.columns)

    jun_cols = []
    jul_cols = []
    aug_cols = []
    sep_cols = []
    common_cols = []
    date_cols = []
    
    for i in range(0, col_len):
        if any(pd.Series(df.columns[i]).str.contains('_6|jun')):
            jun_cols.append(df.columns[i])
        elif any(pd.Series(df.columns[i]).str.contains('_7|jul')):
            jul_cols.append(df.columns[i])
        elif any(pd.Series(df.columns[i]).str.contains('_8|aug')):
            aug_cols.append(df.columns[i])
        elif any(pd.Series(df.columns[i]).str.contains('_9|sep')):
            sep_cols.append(df.columns[i])
        else:
            common_cols.append(df.columns[i])
        
        if any(pd.Series(df.columns[i]).str.contains('date')):
            date_cols.append(df.columns[i])
            
    return jun_cols,jul_cols,aug_cols,sep_cols,common_cols,date_cols

In [ ]:
'''
Input(s)    : 1 list
Output(s)   : 4 lists  
Descripción :
- Este método obtiene la lista de columnas como entrada y las divide en varias subcategorías, como call_usage, recharge_columns,
   columnas relacionadas entrantes y salientes
- Luego devuelve las listas de columnas como salida para cada una de estas subcategorías.
'''
def get_cols_sub_split(col_list):
    call_usage_cols = []
    recharge_cols = []
    ic_usage_cols = []
    og_usage_cols = []

    call_usage_search_for = ['og','ic','mou']

    for i in range(0, len(col_list)):
        if any(pd.Series(col_list[i]).str.contains('|'.join(['rech','rch']))):
            recharge_cols.append(col_list[i])
        elif any(pd.Series(col_list[i]).str.contains('|'.join(call_usage_search_for))):
            call_usage_cols.append(col_list[i])

        if any(pd.Series(col_list[i]).str.contains('ic')):
            ic_usage_cols.append(col_list[i])
        elif any(pd.Series(col_list[i]).str.contains('og')):
            og_usage_cols.append(col_list[i])
            
    return call_usage_cols,recharge_cols,ic_usage_cols,og_usage_cols            

Es objetivo tener solo los registros de clientes de alto valor.


---



Para filtrar los registros de clientes de alto valor:

* Se deriva la columna de la cantidad promedio de recarga para los meses de junio y julio (la fase buena)
* Se toma solo los registros que superen el percentil 70 del monto promedio de recarga.
* Descarta la columna recién derivada que no es necesaria.
* Imprime el recuento de filas y columnas del nuevo marco de datos filtrado.

In [ ]:
# Obtenga el monto promedio de recarga para los meses 6 y 7
data['avg_rech_amt_6_7'] = ( data['total_rech_amt_6'] + data['total_rech_amt_7'] ) / 2

# Obtener los datos superiores al percentil 70 de la cantidad promedio de recarga
data = data.loc[(data['avg_rech_amt_6_7'] > np.percentile(data['avg_rech_amt_6_7'], 70))]

# Quitar la columna promedio
data.drop(['avg_rech_amt_6_7'], axis=1, inplace=True)

print(data.shape)

(29979, 226)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### CLASIFICAR LOS CLIENTES EN CANCELADOS O NO

Para etiquetar a los clientes cancelados:
* Tome las columnas como 'total_ic_mou_9', 'total_og_mou_9', 'vol_2g_mb_9', 'vol_3g_mb_9'
* Encuentre la suma de los valores de las columnas anteriores para cada registro
* Etiquete el registro del cliente como Churned (1) si el valor de la suma es 0 sino Non-Churned (0) y obtenga la columna Churn

In [ ]:
tag_churn_cols = ['total_ic_mou_9', 'total_og_mou_9', 'vol_2g_mb_9', 'vol_3g_mb_9']
data['churn'] = np.where(data[tag_churn_cols].sum(axis=1) == 0, 1, 0 )

In [ ]:
# Ahora verifique el recuento de clientes abandonados en el conjunto de datos obtenido.
data['churn'].value_counts()

0    27390
1     2589
Name: churn, dtype: int64

In [ ]:
print('Churn Rate : {0}%'.format(round(((sum(data['churn'])/len(data['churn']))*100),2)))

Churn Rate : 8.64%


La tasa de abandono es inferior al 10 % de los datos generales disponibles. Esto indica que necesitaríamos manejar el desequilibrio de clases en este problema de clasificación.

In [ ]:
jun_cols, jul_cols, aug_cols, sep_cols, common_cols, date_cols = get_cols_split(data)

data.drop(sep_cols, axis=1, inplace=True)

Verificar y eliminar las columnas no deseadas:

---


Ahora podemos verificar las columnas con menos información y eliminarlas si no son necesarias.

In [ ]:
# Get the unique count
for col in data.columns:
    print(col, len(data[col].unique()))

mobile_number 29979
circle_id 1
loc_og_t2o_mou 2
std_og_t2o_mou 2
loc_ic_t2o_mou 2
last_date_of_month_6 1
last_date_of_month_7 2
last_date_of_month_8 2
arpu_6 29230
arpu_7 29228
arpu_8 28376
onnet_mou_6 18806
onnet_mou_7 18934
onnet_mou_8 17598
offnet_mou_6 22441
offnet_mou_7 22639
offnet_mou_8 21500
roam_ic_mou_6 4338
roam_ic_mou_7 3649
roam_ic_mou_8 3654
roam_og_mou_6 5174
roam_og_mou_7 4431
roam_og_mou_8 4382
loc_og_t2t_mou_6 11150
loc_og_t2t_mou_7 11152
loc_og_t2t_mou_8 10770
loc_og_t2m_mou_6 16739
loc_og_t2m_mou_7 16865
loc_og_t2m_mou_8 16155
loc_og_t2f_mou_6 3252
loc_og_t2f_mou_7 3267
loc_og_t2f_mou_8 3124
loc_og_t2c_mou_6 1659
loc_og_t2c_mou_7 1750
loc_og_t2c_mou_8 1731
loc_og_mou_6 19677
loc_og_mou_7 19867
loc_og_mou_8 18872
std_og_t2t_mou_6 12773
std_og_t2t_mou_7 12982
std_og_t2t_mou_8 11781
std_og_t2m_mou_6 14512
std_og_t2m_mou_7 14583
std_og_t2m_mou_8 13320
std_og_t2f_mou_6 1774
std_og_t2f_mou_7 1715
std_og_t2f_mou_8 1627
std_og_t2c_mou_6 2
std_og_t2c_mou_7 2
std_og_t2c_mou_

Podemos observar del resultado anterior que:

* Columna Unique_ID - mobile_number
* Columnas de menor información
    * circle_id
    * last_date_of_month_6
    * last_date_of_month_7
    * last_date_of_month_8
    * loc_og_t2o_mou
    * std_og_t2o_mou
    * loc_ic_t2o_mou
    * std_og_t2c_mou_6
    * std_og_t2c_mou_7
    * std_og_t2c_mou_8
    * std_ic_t2o_mou_6
    * std_ic_t2o_mou_7
    * std_ic_t2o_mou_8
    
Echemos un vistazo a algunos registros de muestra de las columnas anteriores.

In [ ]:
data[['mobile_number','circle_id','last_date_of_month_6','last_date_of_month_7','last_date_of_month_8',\
           'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou','std_og_t2c_mou_6','std_og_t2c_mou_7','std_og_t2c_mou_8',\
           'std_ic_t2o_mou_6','std_ic_t2o_mou_7','std_ic_t2o_mou_8']].head(5)

,mobile_number,circle_id,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8
7,7000701601,109,6/30/2014,7/31/2014,8/31/2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,7001524846,109,6/30/2014,7/31/2014,8/31/2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,7002191713,109,6/30/2014,7/31/2014,8/31/2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,7000875565,109,6/30/2014,7/31/2014,8/31/2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,7000187447,109,6/30/2014,7/31/2014,8/31/2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Podemos eliminar las columnas enumeradas anteriormente del conjunto de datos.
data.drop(['mobile_number','circle_id','last_date_of_month_6','last_date_of_month_7','last_date_of_month_8',\
           'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou','std_og_t2c_mou_6','std_og_t2c_mou_7','std_og_t2c_mou_8',\
           'std_ic_t2o_mou_6','std_ic_t2o_mou_7','std_ic_t2o_mou_8'], axis=1, inplace=True)

Now let's check for the recharge columns value.

In [ ]:
# Valor de las columnas de recarga
data[['total_rech_data_6','av_rech_amt_data_6','max_rech_data_6']].head()

,total_rech_data_6,av_rech_amt_data_6,max_rech_data_6
7,NaN,NaN,NaN
8,NaN,NaN,NaN
13,NaN,NaN,NaN
16,NaN,NaN,NaN
17,NaN,NaN,NaN


Encontramos que el monto de recarga promedio para las columnas de datos en realidad representa el monto total en lugar del valor promedio. Así que cambiaremos el nombre de las columnas al nombre propio.

In [ ]:
data = data.rename(columns={'av_rech_amt_data_6':'total_rech_amt_data_6',
                            'av_rech_amt_data_7':'total_rech_amt_data_7',
                            'av_rech_amt_data_8':'total_rech_amt_data_8'})

### MANEJO DE VALORES FALTANTES

In [ ]:
# A continuación, verificaremos el recuento de valores faltantes en el conjunto de datos y 
# enumeraremos las columnas con los valores faltantes.
df = data.isnull().sum().reset_index(name='missing_cnt')
df.loc[df['missing_cnt']>0].sort_values('missing_cnt', ascending=False)

,index,missing_cnt
114,count_rech_2g_6,18592
108,total_rech_data_6,18592
120,total_rech_amt_data_6,18592
129,arpu_3g_6,18592
111,max_rech_data_6,18592
...,...,...
46,spl_og_mou_7,303
43,isd_og_mou_7,303
40,std_og_mou_7,303
100,date_of_last_rech_7,114


Ahora utilizaremos la función auxiliar **get_cols_split** para obtener las categorías de columnas.


---



In [ ]:
jun_cols, jul_cols, aug_cols, sep_cols, common_cols, date_cols = get_cols_split(data)

Del mismo modo, la lista de columnas del mes va a la función auxiliar **get_cols_sub_split** para obtener subcategorías de las columnas.


---



In [ ]:
# Get the columns sub split for each months
jun_call_usage_cols, jun_recharge_cols, jun_ic_usage_cols, jun_og_usage_cols = get_cols_sub_split(jun_cols)
jul_call_usage_cols, jul_recharge_cols, jul_ic_usage_cols, jul_og_usage_cols = get_cols_sub_split(jul_cols)
aug_call_usage_cols, aug_recharge_cols, aug_ic_usage_cols, aug_og_usage_cols = get_cols_sub_split(aug_cols)

There are few missing values which we will start filling in one by one. 

fb_user and night_pack_user columns are of nominal type 0 and 1. Since missing values could be of another type, imputing them as 2.

In [ ]:
# Completar los valores faltantes de las columnas de usuario de fb y night pack como 2,
# ya que este podría ser otro tipo que se marcó como NA
cols_6 = ['fb_user_6','night_pck_user_6']
cols_7 = ['fb_user_7','night_pck_user_7']
cols_8 = ['fb_user_8','night_pck_user_8']

data[cols_6] = data[cols_6].fillna(2)
data[cols_7] = data[cols_7].fillna(2)
data[cols_8] = data[cols_8].fillna(2)

Los valores que faltan para el siguiente conjunto de columnas parecen ser datos no disponibles. Entonces se imponen a 0.

In [ ]:
# Los valores que faltan para el siguiente conjunto de columnas parecen ser datos no disponibles. Entonces se imponen a 0.
cols_6 = ['count_rech_3g_6','max_rech_data_6','total_rech_amt_data_6','arpu_3g_6','total_rech_data_6','count_rech_2g_6','arpu_2g_6']
cols_7 = ['count_rech_3g_7','max_rech_data_7','total_rech_amt_data_7','arpu_3g_7','total_rech_data_7','count_rech_2g_7','arpu_2g_7']
cols_8 = ['count_rech_3g_8','max_rech_data_8','total_rech_amt_data_8','arpu_3g_8','total_rech_data_8','count_rech_2g_8','arpu_2g_8']

data[cols_6] = data[cols_6].fillna(0)
data[cols_7] = data[cols_7].fillna(0)
data[cols_8] = data[cols_8].fillna(0)

data[jun_call_usage_cols] = data[jun_call_usage_cols].fillna(0)
data[jul_call_usage_cols] = data[jul_call_usage_cols].fillna(0)
data[aug_call_usage_cols] = data[aug_call_usage_cols].fillna(0)

In [ ]:
# Se dejan las columnas de fecha como nulas intencionalmente para la ingeniería de características
df = data.isnull().sum().reset_index(name='missing_cnt')
df.loc[df['missing_cnt']>0].sort_values('missing_cnt', ascending=False)

,index,missing_cnt
105,date_of_last_rech_data_6,18592
106,date_of_last_rech_data_7,18327
107,date_of_last_rech_data_8,18238
101,date_of_last_rech_8,594
100,date_of_last_rech_7,114
99,date_of_last_rech_6,62
